In [9]:
# Ontology_V1
# Question Template 1-3: 
# Want to cook spicy food/meal_1,If you do not have pepper, what other objects in the image can be used instead? 

from owlready2 import *
from itertools import chain

onto = get_ontology("http://test.org/onto.owl")

with onto:
    class Food(Thing):
        pass
    class Meat(Food):
        pass
    class Vegetable(Food):
        pass
    class Fruit(Food):
        pass
    class Taste(Thing):
        pass
    
    class Recipe(Thing):
        pass
    class Ingredient(Thing):
        pass
    
    class People(Thing):
        pass
    class Vegetarian(People):
        equivalent_to = [People & Not(Meat)]
        pass
    class NonVegetarian(People):
        equivalent_to = [People & Food]
        pass
    
    # Ontology for Question Template 4
    class Cook(Thing):
        pass
    class Tool(Thing):
        pass
    
    
    class has_for_taste(ObjectProperty):
        domain    = [Food]
        range     = [Taste]  
    class has_ingredient(ObjectProperty):
        domain = [Recipe]
        range = [Food]
    
    # Different taste buds - 9
    spicy = Taste('spicy')
    sweet = Taste('sweet')
    meaty = Taste('meaty')
    pasty = Taste('pasty')
    eggy = Taste('eggy')
    
    # Instance of ingredient
    egg = Food('egg', has_for_taste=[eggy])
    beef = Meat('beef', has_for_taste=[meaty])
    spice_beef = Meat('spice_beef', has_for_taste=[meaty, spicy])
    chicken = Meat('chicken', has_for_taste=[meaty])
    fish = Meat('fish', has_for_taste=[meaty])
    
    pepper = Food('pepper', has_for_taste=[spicy])
    chilli = Food('chilli', has_for_taste=[spicy])

    sugar = Food('sugar', has_for_taste=[sweet])
    honey = Food('honey', has_for_taste=[sweet])
    
    orange = Fruit("orange", has_for_taste=[sweet])
    lemon = Fruit("lemon", has_for_taste=[sweet])

    noodle = Food('noodle', has_for_taste=[pasty])
    spaghetti = Food('spaghetti', has_for_taste=[pasty])
    
    rice = Food('rice')
    corn = Food('corn')
    
    

    
    # Ontology for Question Template 4: Want to boil eggs or fry eggs, what tools shall I use? 
    boil = Cook('boil')
    fry = Cook('fry')
    chop = Cook('chop')
    
    pan = Tool('pan')
    pot = Tool('pot')
    knife = Tool('knife')
    
    vegetarian = Vegetarian('vegetarian')
    nonvegetarian = NonVegetarian('nonvegetarian')
    # QB: is_similar_with does not work properly, because, we need to define name first, 
    #then put them into is_similar_with
    
     
#     rice = Ingredient('rice', is_similar_with=[corn])
#     corn = Ingredient('corn', is_similar_with=[rice])
 
#     noodle = Ingredient('noodle', is_similar_with=[pasta])
#     pasta = Ingredient('pasta', is_similar_with=[noodle])

#     beef = Ingredient('beef', is_similar_with=[chicken])
#     chicken = Ingredient('chicken', is_similar_with=[beef])
#     fish = Ingredient('fish')
    
#     pepper = Ingredient('pepper')
#     chilli = Ingredient('chilli')

#     sugar = Ingredient('sugar', is_similar_with=[honey])
#     honey = Ingredient('honey', is_similar_with=[sugar])
    
#     orange = Fruit("orange", is_similar_with=[lemon])
#     lemon = Fruit("lemon", is_similar_with=[orange])
    
    
    # Instance of recipe - ingredient specification

#      class Ingredient_Specification(Thing):
#         pass
    
    
#     class has_for_taste(ObjectProperty):
#         domain    = [Food, Ingredient_Specification]
#         range     = [Taste]  
#     class has_ingredient(ObjectProperty):
#         domain = [Recipe]
#         range = [Ingredient_Specification]       
#     class recommended_ingredient(ObjectProperty):
#         domain = [Ingredient_Specification]
#         range = [Food]

#     pasta_noodle = Ingredient_Specification('pasta_noodle', has_for_taste=[pasty], recommended_ingredient=[noodle])
#     meat_beef = Ingredient_Specification('meat_beef', has_for_taste=[meaty], recommended_ingredient=[beef])
#     spicy_pepper = Ingredient_Specification('spicy_pepper', has_for_taste=[spicy], recommended_ingredient=[pepper])
#     spicy_noodle = Recipe('spicy_noodle', has_ingredient=[pasta_noodle, meat_beef, spicy_pepper])
    
    spicy_noodle = Recipe('spicy_noodle', has_ingredient=[noodle, beef, pepper])
    fried_rice = Recipe('fried_rice', has_ingredient=[rice, chicken, chilli])
    sweet_cake = Recipe('sweet_cake', has_ingredient=[sugar, orange])

    
    # QB - has_ingredient can include the similar ingredients 

In [10]:
import numpy as np
import pandas as pd
from pprint import pprint

# --- First method set, recursive methods to turn objects and relations into some sort of consistent format ---
# --- Output: list of tuples to represent the entity and all its relationships including a depth tracker ---

# recursive method to get class hierarchy
def get_types_hierarchy(x):
    if issubclass(x, Thing):
        # to filter out Thing's, uncomment below
        # if x is not Thing:
        return [x, *get_types_hierarchy(x.__bases__[0])]
    return []

# gets a tuple representing a relationship (recursive with get_object_pairs)
def get_relation_pairs(o, depth=1):
    if depth>=10:
        print('Warning, depth of 10 hit, check for cycles')
        return []
    return [('relation', p, get_object_pairs(getattr(o, p._name), depth=depth+1)) 
    for p in o.get_properties() if ObjectPropertyClass]

# gets a tuple representing a pair of objects (recursive with get_relation_pairs)
def get_object_pairs(objects, limit_domain=[], depth=1):
    return [(
        'entity',
        get_types_hierarchy(type(x)),
        x,
        get_relation_pairs(x, depth))
        for x in objects if len(limit_domain)==0 or any([isinstance(x, y) for y in limit_domain])]


# --- Second method set, recursive method sets to unroll the hierarchy, ---
# --- Output: collapsed list of dicts with each level of entity/relationship numbered as e1/r1, e2/r2 etc. ---

# works with dictify to flatten out the hierarchy to a one-dict per entity/relationship combination including recurse
def dict_collapse(e, depth=1):
    has_relationships = len(e[3])>0
    if has_relationships:
        return [{f'e{depth}': e[0:3], f'r{depth}': r[0:2], 'n': dictify(r[2], depth+1)} for r in e[3]]
    else:
        return [{f'e{depth}': e[0:3], f'r{depth}': None, 'n': []}]

# removes the "n" column which is used to indicate another level of recursion
def drop_n(d):
    return {k:v for k,v in d.items() if k!='n'}

# helper to flatten nested lists
def flatten(l):
    return list(chain.from_iterable(l))

# Removes a single level of recursion then calls itself again
def flatten_dicts(dict_list):
    # handle case of no next level and case of having a next level then recurse
    recursed_set = [{**drop_n(e), **n} for e in dict_list for n in e['n']]
    if any(['n' in e for e in recursed_set]):
        recursed_set = flatten_dicts(recursed_set)
    return recursed_set + \
        [drop_n(e) for e in dict_list if len(e['n'])==0]

# controller method to orchestrate the other 2
def dictify(object_pairs, depth=1):
    if depth >= 10:
        print('Warning, depth of 10 hit, check for cycles')
        return []
    collapsed_dicts = flatten([dict_collapse(e, depth) for e in object_pairs])
    return flatten_dicts(collapsed_dicts)

# --- Third method set, no more recursion, these methods are just helpful to coerce to a table-like format, could easily write your own methods instead :) ---
# --- Output: dictionary with an entity or list of classes per column making it easier to filter and work with ---

# convert the whole ontology to a table format
from collections import ChainMap
def tablify(object_tuple_dict):
    def generate_entity_dict(k, v):
        if v is None:
            return {}
        elif v[0]=='entity':
            return {f'{k}_classes': v[1], f'{k}_entity': v[2]}
        else:
            return {f'{k}_relation': v[1]}
        
    return [dict(ChainMap(*[generate_entity_dict(k,v) for k,v in x.items()])) for x in object_tuple_dict]

# convert all the entities and classes to strings for easy sentence generation
def stringify_table(dict_table_format):
    return [{k: [x.name for x in v] if isinstance(v, list) else v._name for k,v in x.items()} for x in dict_table_format]


In [11]:
# these 2 do the main bulk of the work, you could happily write a function to process object_tuple_dict directly
object_pairs = get_object_pairs(onto.individuals())
object_tuple_dict = dictify(object_pairs)

# or use these to get a table sort of format that we can do further processing on to generate questions!
dict_table_format = tablify(object_tuple_dict)
string_table = stringify_table(dict_table_format)

table_cols_threelevels = ['e1_classes', 'e1_entity', 'r1_relation', 'e2_classes', 'e2_entity', 'r2_relation', 'e3_classes', 'e3_entity']
# entity_df = pd.DataFrame(dict_table_format)[table_cols_threelevels]
# entity_df
string_df = pd.DataFrame(string_table)[table_cols_threelevels]
string_df

,e1_classes,e1_entity,r1_relation,e2_classes,e2_entity,r2_relation,e3_classes,e3_entity
0,"[Vegetable, Food, Thing]",tomato,original_from,"[Country, Thing]",australia,NaN,NaN,NaN
1,"[Vegetable, Food, Thing]",tomato,has_for_taste,"[Taste, Thing]",juicy,NaN,NaN,NaN
2,"[Vegetable, Food, Thing]",tomato,has_for_taste,"[Taste, Thing]",sweet,NaN,NaN,NaN
3,"[Vegetable, Food, Thing]",broccoli,has_for_taste,"[Taste, Thing]",juicy,NaN,NaN,NaN
4,"[Vegetable, Food, Thing]",broccoli,has_for_taste,"[Taste, Thing]",sweet,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
144,"[Tool, Thing]",pan,NaN,NaN,NaN,NaN,NaN,NaN
145,"[Tool, Thing]",pot,NaN,NaN,NaN,NaN,NaN,NaN
146,"[Tool, Thing]",knife,NaN,NaN,NaN,NaN,NaN,NaN
147,"[Vegetarian, People, Thing]",vegetarian,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# these 2 do the main bulk of the work, you could happily write a function to process object_tuple_dict directly
object_pairs = get_object_pairs(onto.individuals())
object_tuple_dict = dictify(object_pairs)

# or use these to get a table sort of format that we can do further processing on to generate questions!
dict_table_format = tablify(object_tuple_dict)
string_table = stringify_table(dict_table_format)

table_cols_threelevels = ['e1_classes', 'e1_entity', 'r1_relation', 'e2_classes', 'e2_entity', 'r2_relation', 'e3_classes', 'e3_entity']
# entity_df = pd.DataFrame(dict_table_format)[table_cols_threelevels]
# entity_df
string_df = pd.DataFrame(string_table)[table_cols_threelevels]
string_df

,e1_classes,e1_entity,r1_relation,e2_classes,e2_entity,r2_relation,e3_classes,e3_entity
0,"[Vegetable, Food, Thing]",tomato,original_from,"[Country, Thing]",australia,NaN,NaN,NaN
1,"[Vegetable, Food, Thing]",tomato,has_for_taste,"[Taste, Thing]",juicy,NaN,NaN,NaN
2,"[Vegetable, Food, Thing]",tomato,has_for_taste,"[Taste, Thing]",sweet,NaN,NaN,NaN
3,"[Vegetable, Food, Thing]",broccoli,has_for_taste,"[Taste, Thing]",juicy,NaN,NaN,NaN
4,"[Vegetable, Food, Thing]",broccoli,has_for_taste,"[Taste, Thing]",sweet,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
144,"[Tool, Thing]",pan,NaN,NaN,NaN,NaN,NaN,NaN
145,"[Tool, Thing]",pot,NaN,NaN,NaN,NaN,NaN,NaN
146,"[Tool, Thing]",knife,NaN,NaN,NaN,NaN,NaN,NaN
147,"[Vegetarian, People, Thing]",vegetarian,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
recipe_filter = string_df.e1_classes.apply(lambda x: 'Recipe' in x)
recipe_name = string_df[recipe_filter]
recipe_name

,e1_classes,e1_entity,r1_relation,e2_classes,e2_entity,r2_relation,e3_classes,e3_entity
100,"[Recipe, Thing]",spicy_noodle,has_ingredient,"[Food, Thing]",noodle,has_for_taste,"[Taste, Thing]",pasty
101,"[Recipe, Thing]",spicy_noodle,has_ingredient,"[Meat, Food, Thing]",beef,has_for_taste,"[Taste, Thing]",meaty
102,"[Recipe, Thing]",spicy_noodle,has_ingredient,"[Food, Thing]",pepper,has_for_taste,"[Taste, Thing]",spicy
103,"[Recipe, Thing]",fried_rice,has_ingredient,"[Meat, Food, Thing]",chicken,has_for_taste,"[Taste, Thing]",meaty
104,"[Recipe, Thing]",fried_rice,has_ingredient,"[Vegetable, Food, Thing]",chilli,original_from,"[Country, Thing]",mexica
105,"[Recipe, Thing]",fried_rice,has_ingredient,"[Vegetable, Food, Thing]",chilli,has_for_taste,"[Taste, Thing]",spicy
106,"[Recipe, Thing]",fried_rice,has_ingredient,"[Vegetable, Food, Thing]",chilli,has_color,"[Color, Thing]",green
107,"[Recipe, Thing]",fried_rice,has_ingredient,"[Vegetable, Food, Thing]",chilli,has_color,"[Color, Thing]",red
108,"[Recipe, Thing]",fried_rice,has_ingredient,"[Vegetable, Food, Thing]",chilli,has_color,"[Color, Thing]",yellow
109,"[Recipe, Thing]",fried_rice,has_ingredient,"[Food, Thing]",rice,NaN,NaN,NaN


In [14]:
# or can make a string df use the "string_table" for sentence generation purposes

# exp = string_df.explode('e2_classes').explode('e1_classes')

some_questions = (
    ('Want to cook ' + recipe_name.e1_entity + ', if we do not have ' + recipe_name.e2_entity + ', what other food in the image can be used instead?').unique().tolist() )
some_questions

['Want to cook spicy_noodle, if we do not have noodle, what other food in the image can be used instead?',
 'Want to cook spicy_noodle, if we do not have beef, what other food in the image can be used instead?',
 'Want to cook spicy_noodle, if we do not have pepper, what other food in the image can be used instead?',
 'Want to cook fried_rice, if we do not have chicken, what other food in the image can be used instead?',
 'Want to cook fried_rice, if we do not have chilli, what other food in the image can be used instead?',
 'Want to cook fried_rice, if we do not have rice, what other food in the image can be used instead?',
 'Want to cook sweet_cake, if we do not have sugar, what other food in the image can be used instead?',
 'Want to cook sweet_cake, if we do not have orange, what other food in the image can be used instead?']

In [15]:
some_questions = (
    ('Want to cook ' + recipe_name.e1_entity + ', if we do not have ' + recipe_name.e2_entity + ', can we use three other things to replace it?').unique().tolist() )
from pprint import pprint
pprint(some_questions)

['Want to cook spicy_noodle, if we do not have noodle, can we use three other '
 'things to replace it?',
 'Want to cook spicy_noodle, if we do not have beef, can we use three other '
 'things to replace it?',
 'Want to cook spicy_noodle, if we do not have pepper, can we use three other '
 'things to replace it?',
 'Want to cook fried_rice, if we do not have chicken, can we use three other '
 'things to replace it?',
 'Want to cook fried_rice, if we do not have chilli, can we use three other '
 'things to replace it?',
 'Want to cook fried_rice, if we do not have rice, can we use three other '
 'things to replace it?',
 'Want to cook sweet_cake, if we do not have sugar, can we use three other '
 'things to replace it?',
 'Want to cook sweet_cake, if we do not have orange, can we use three other '
 'things to replace it?']


In [16]:
# Generate questions_1-3 from table
# If I want to boil eggs or fry eggs , which tools in the image can be used? 
# If I want to chop an onion, which tool in the image best suit?
cook_filter = string_df.e1_classes.apply(lambda x: 'Cook' in x)
cook_name = string_df[cook_filter]
cook_name

,e1_classes,e1_entity,r1_relation,e2_classes,e2_entity,r2_relation,e3_classes,e3_entity
141,"[Cook, Thing]",boil,NaN,NaN,NaN,NaN,NaN,NaN
142,"[Cook, Thing]",fry,NaN,NaN,NaN,NaN,NaN,NaN
143,"[Cook, Thing]",chop,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
food_filter = string_df.e1_classes.apply(lambda x: 'Food' in x)
food_name = string_df[food_filter]
food_name

,e1_classes,e1_entity,r1_relation,e2_classes,e2_entity,r2_relation,e3_classes,e3_entity
0,"[Vegetable, Food, Thing]",tomato,original_from,"[Country, Thing]",australia,NaN,NaN,NaN
1,"[Vegetable, Food, Thing]",tomato,has_for_taste,"[Taste, Thing]",juicy,NaN,NaN,NaN
2,"[Vegetable, Food, Thing]",tomato,has_for_taste,"[Taste, Thing]",sweet,NaN,NaN,NaN
3,"[Vegetable, Food, Thing]",broccoli,has_for_taste,"[Taste, Thing]",juicy,NaN,NaN,NaN
4,"[Vegetable, Food, Thing]",broccoli,has_for_taste,"[Taste, Thing]",sweet,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
97,"[Food, Thing]",honey,has_for_taste,"[Taste, Thing]",sweet,NaN,NaN,NaN
98,"[Food, Thing]",noodle,has_for_taste,"[Taste, Thing]",pasty,NaN,NaN,NaN
99,"[Food, Thing]",spaghetti,has_for_taste,"[Taste, Thing]",pasty,NaN,NaN,NaN
139,"[Food, Thing]",rice,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Question Template_4
# If I want to boil eggs or fry eggs , which tools in the image can be used? 
# If I want to chop an onion, which tool in the image best suit?
some_questions = (
    ('If I want to  ' + cook_name.e1_entity + ' an egg ' +', which tools in the image can be used?').unique().tolist() )
from pprint import pprint
pprint(some_questions)

['If I want to  boil an egg , which tools in the image can be used?',
 'If I want to  fry an egg , which tools in the image can be used?',
 'If I want to  chop an egg , which tools in the image can be used?']


In [19]:
# pd.merge(cook_name, food_name)
cook_name_copy = cook_name.copy()
food_name_copy = food_name.copy()
cook_name_copy['key'] = 0
food_name_copy['key'] = 0

food_cooking_df = food_name_copy.merge(cook_name_copy, on='key', how='outer', suffixes=('_food', '_cook'))
food_cooking_df

,e1_classes_food,e1_entity_food,r1_relation_food,e2_classes_food,e2_entity_food,r2_relation_food,e3_classes_food,e3_entity_food,key,e1_classes_cook,e1_entity_cook,r1_relation_cook,e2_classes_cook,e2_entity_cook,r2_relation_cook,e3_classes_cook,e3_entity_cook
0,"[Vegetable, Food, Thing]",tomato,original_from,"[Country, Thing]",australia,NaN,NaN,NaN,0,"[Cook, Thing]",boil,NaN,NaN,NaN,NaN,NaN,NaN
1,"[Vegetable, Food, Thing]",tomato,original_from,"[Country, Thing]",australia,NaN,NaN,NaN,0,"[Cook, Thing]",fry,NaN,NaN,NaN,NaN,NaN,NaN
2,"[Vegetable, Food, Thing]",tomato,original_from,"[Country, Thing]",australia,NaN,NaN,NaN,0,"[Cook, Thing]",chop,NaN,NaN,NaN,NaN,NaN,NaN
3,"[Vegetable, Food, Thing]",tomato,has_for_taste,"[Taste, Thing]",juicy,NaN,NaN,NaN,0,"[Cook, Thing]",boil,NaN,NaN,NaN,NaN,NaN,NaN
4,"[Vegetable, Food, Thing]",tomato,has_for_taste,"[Taste, Thing]",juicy,NaN,NaN,NaN,0,"[Cook, Thing]",fry,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,"[Food, Thing]",rice,NaN,NaN,NaN,NaN,NaN,NaN,0,"[Cook, Thing]",fry,NaN,NaN,NaN,NaN,NaN,NaN
302,"[Food, Thing]",rice,NaN,NaN,NaN,NaN,NaN,NaN,0,"[Cook, Thing]",chop,NaN,NaN,NaN,NaN,NaN,NaN
303,"[Food, Thing]",corn,NaN,NaN,NaN,NaN,NaN,NaN,0,"[Cook, Thing]",boil,NaN,NaN,NaN,NaN,NaN,NaN
304,"[Food, Thing]",corn,NaN,NaN,NaN,NaN,NaN,NaN,0,"[Cook, Thing]",fry,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Question Template_4
# If I want to boil eggs or fry eggs , which tools in the image can be used? 
# If I want to chop an onion, which tool in the image best suit?

some_questions = (
    ('If I want to  ' + food_cooking_df.e1_entity_cook + ' ' + food_cooking_df.e1_entity_food +', which tools in the image can be used?').unique().tolist() )
from pprint import pprint

some_questions
# pprint(some_questions[:5])
# pprint(np.random.choice(some_questions, 5))

['If I want to  boil tomato, which tools in the image can be used?',
 'If I want to  fry tomato, which tools in the image can be used?',
 'If I want to  chop tomato, which tools in the image can be used?',
 'If I want to  boil broccoli, which tools in the image can be used?',
 'If I want to  fry broccoli, which tools in the image can be used?',
 'If I want to  chop broccoli, which tools in the image can be used?',
 'If I want to  boil bean, which tools in the image can be used?',
 'If I want to  fry bean, which tools in the image can be used?',
 'If I want to  chop bean, which tools in the image can be used?',
 'If I want to  boil chilli, which tools in the image can be used?',
 'If I want to  fry chilli, which tools in the image can be used?',
 'If I want to  chop chilli, which tools in the image can be used?',
 'If I want to  boil carrot, which tools in the image can be used?',
 'If I want to  fry carrot, which tools in the image can be used?',
 'If I want to  chop carrot, which tool

In [21]:
# Question Template 6 - Missing an instance/subclass 
# What food/ingredient is missing to cook beef noodle? In the image: we have beef, tomato, water, but do not have noodle.
# Question: What ingredient is missing to cook the spicy beef noodle?  
some_questions = (
    ('Which ingredient is missing to' + ' ' + recipe_name.e1_entity + '?').unique().tolist() )
from pprint import pprint
pprint(some_questions)

['Which ingredient is missing to spicy_noodle?',
 'Which ingredient is missing to fried_rice?',
 'Which ingredient is missing to sweet_cake?']


In [22]:
# pd.merge(cook_name, recipe_name)
cook_name_copy = cook_name.copy()
recipe_name_copy = recipe_name.copy()
cook_name_copy['key'] = 0
recipe_name_copy['key'] = 0
missing_cook_df = recipe_name_copy.merge(cook_name_copy, on='key', how='outer', suffixes=('_recipe', '_cook'))
missing_cook_df

,e1_classes_recipe,e1_entity_recipe,r1_relation_recipe,e2_classes_recipe,e2_entity_recipe,r2_relation_recipe,e3_classes_recipe,e3_entity_recipe,key,e1_classes_cook,e1_entity_cook,r1_relation_cook,e2_classes_cook,e2_entity_cook,r2_relation_cook,e3_classes_cook,e3_entity_cook
0,"[Recipe, Thing]",spicy_noodle,has_ingredient,"[Food, Thing]",noodle,has_for_taste,"[Taste, Thing]",pasty,0,"[Cook, Thing]",boil,NaN,NaN,NaN,NaN,NaN,NaN
1,"[Recipe, Thing]",spicy_noodle,has_ingredient,"[Food, Thing]",noodle,has_for_taste,"[Taste, Thing]",pasty,0,"[Cook, Thing]",fry,NaN,NaN,NaN,NaN,NaN,NaN
2,"[Recipe, Thing]",spicy_noodle,has_ingredient,"[Food, Thing]",noodle,has_for_taste,"[Taste, Thing]",pasty,0,"[Cook, Thing]",chop,NaN,NaN,NaN,NaN,NaN,NaN
3,"[Recipe, Thing]",spicy_noodle,has_ingredient,"[Meat, Food, Thing]",beef,has_for_taste,"[Taste, Thing]",meaty,0,"[Cook, Thing]",boil,NaN,NaN,NaN,NaN,NaN,NaN
4,"[Recipe, Thing]",spicy_noodle,has_ingredient,"[Meat, Food, Thing]",beef,has_for_taste,"[Taste, Thing]",meaty,0,"[Cook, Thing]",fry,NaN,NaN,NaN,NaN,NaN,NaN
5,"[Recipe, Thing]",spicy_noodle,has_ingredient,"[Meat, Food, Thing]",beef,has_for_taste,"[Taste, Thing]",meaty,0,"[Cook, Thing]",chop,NaN,NaN,NaN,NaN,NaN,NaN
6,"[Recipe, Thing]",spicy_noodle,has_ingredient,"[Food, Thing]",pepper,has_for_taste,"[Taste, Thing]",spicy,0,"[Cook, Thing]",boil,NaN,NaN,NaN,NaN,NaN,NaN
7,"[Recipe, Thing]",spicy_noodle,has_ingredient,"[Food, Thing]",pepper,has_for_taste,"[Taste, Thing]",spicy,0,"[Cook, Thing]",fry,NaN,NaN,NaN,NaN,NaN,NaN
8,"[Recipe, Thing]",spicy_noodle,has_ingredient,"[Food, Thing]",pepper,has_for_taste,"[Taste, Thing]",spicy,0,"[Cook, Thing]",chop,NaN,NaN,NaN,NaN,NaN,NaN
9,"[Recipe, Thing]",fried_rice,has_ingredient,"[Meat, Food, Thing]",chicken,has_for_taste,"[Taste, Thing]",meaty,0,"[Cook, Thing]",boil,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Question Template 6 - Missing an instance/subclass 
# What food/ingredient is missing to cook beef noodle? In the image: we have beef, tomato, water, but do not have noodle.
# Question: What ingredient is missing to cook the spicy beef noodle?  
some_questions = (
    ('Which ingredient is missing to ' + missing_cook_df.e1_entity_cook + ' ' + missing_cook_df.e1_entity_recipe + '?').unique().tolist() )
from pprint import pprint
pprint(some_questions)

['Which ingredient is missing to boil spicy_noodle?',
 'Which ingredient is missing to fry spicy_noodle?',
 'Which ingredient is missing to chop spicy_noodle?',
 'Which ingredient is missing to boil fried_rice?',
 'Which ingredient is missing to fry fried_rice?',
 'Which ingredient is missing to chop fried_rice?',
 'Which ingredient is missing to boil sweet_cake?',
 'Which ingredient is missing to fry sweet_cake?',
 'Which ingredient is missing to chop sweet_cake?']


In [24]:
# Question Template 7 - Missing a tool to achieve a goal 
# What tool in the image is missing to cook a noodle?
# In the image, we have beef, tomato, water, but do not have a pot, the achieve the action cook a noodle. 
# Answer: We need a pot to finish this task, rather than food or pan as the answer. 
some_questions = (
    ('Which tool is missing to cook' + ' ' + recipe_name.e1_entity + '?').unique().tolist() )
from pprint import pprint
pprint(some_questions)

['Which tool is missing to cook spicy_noodle?',
 'Which tool is missing to cook fried_rice?',
 'Which tool is missing to cook sweet_cake?']


In [25]:
people_filter = string_df.e1_classes.apply(lambda x: 'People' in x)
people_name = string_df[people_filter]
people_name

,e1_classes,e1_entity,r1_relation,e2_classes,e2_entity,r2_relation,e3_classes,e3_entity
147,"[Vegetarian, People, Thing]",vegetarian,NaN,NaN,NaN,NaN,NaN,NaN
148,"[NonVegetarian, People, Thing]",nonvegetarian,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Template 5  
# In the image, all the food are meat.
# Question: Can we use the food in the image to make a meal for vegetarian people? 
# In the image, have both meat and vegetables.
# Question: which food in the image can be used to cook a meal for vegetarian people? 
some_questions = (
    ('Can we use the food in the image to make a meal for' + ' ' + people_name.e1_entity + ' people?').unique().tolist() )
from pprint import pprint
pprint(some_questions)

['Can we use the food in the image to make a meal for vegetarian people?',
 'Can we use the food in the image to make a meal for nonvegetarian people?']
